# Saucisse

In [ ]:
# ELY BP COMPRESSOR VANNE MP HP
# Probleme :  masse volumique Arondie       t = huge                                                                   # Aucun problème rescencé

import CoolProp

def fct_ELY_prod(time_tick_sim_sec, ELY_prod) :
    if time_tick_sim_sec == 1 :
        ELY_prod = {"tension_cell_v":2.7,
                    "puissance_kw":1300*10,
                    "pression_bar":40,
                    "debit_kgh":0,
                    "temp_out_h2_k":273.15+30,
                    "valve_sortie_consigne":1,
                    "total_h2_produced_kg":0,
                    "energie_spent_kwh":0}

    # Simulation loop
    
    ELY_prod["debit_kgh"] = ELY_prod["valve_sortie_consigne"] * ELY_prod["puissance_kw"]*pow(10,3) / (2*96485*ELY_prod["tension_cell_v"]) * 2.02*3.6    # 2.02 g/mol       0.0899 kg/Nm3       3.6 = s to h & g to kg

    ELY_prod["energie_spent_kwh"] = ELY_prod["energie_spent_kwh"] + ELY_prod["valve_sortie_consigne"]*ELY_prod["puissance_kw"]/3600

    ELY_prod["total_h2_produced_kg"] = ELY_prod["total_h2_produced_kg"] + ELY_prod["debit_kgh"]/3600

    return ELY_prod

def fct_stockage(time_tick_sim_sec, stockage, p_init, v_init, q_init, var_stk_in_q_kgh) :
    def fct_stockage_init(p_init, v_init, q_init) :
        var_stk_temp_kelv = 273.14+20                   # Variable VARIABLE 20 = T ambiant
        var_stk_p_bar = p_init                          # Variable VARIABLE
        var_stk_v_max_m3 = v_init                       # Variable VARIABLE
        var_stk_q_max_kgh = q_init                      # Variable VARIABLE

        var_stk_masse_kg = 1/pow(10,6)                  # Variable VARIABLE == 0
        var_stk_roh_kgm3 = var_stk_masse_kg/var_stk_v_max_m3
        var_stk_u_eint = CoolProp.CoolProp.PropsSI('U','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
        var_stk_h_ent = CoolProp.CoolProp.PropsSI('H','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
        var_stk_y_dan = var_stk_masse_kg*var_stk_u_eint
        var_stk_h_ent = -var_stk_u_eint

        stockage_init = {"volume_max_m3":var_stk_v_max_m3,                        # Variable VARIABLE
                        "debit_max_kgh":var_stk_q_max_kgh,                        # Variable VARIABLE
                        "masse_stocked_kg":var_stk_masse_kg,
                        "temp_kelv":var_stk_temp_kelv,
                        "pression_bar":var_stk_p_bar,
                        "masse_volumique_kgm3":var_stk_roh_kgm3,
                        "energie_interne_J":var_stk_u_eint,
                        "entalpie_J":var_stk_h_ent,
                        "delta_entalpie":var_stk_y_dan,
                        "valve_sortie_consigne":0,                                # Variable commande
                        "debit_sortie_kgh":0}
        return stockage_init

    if time_tick_sim_sec == 1 :             # Initialise avec données de base puis change en fonction des entrées sorties
        stockage = fct_stockage_init(p_init, v_init, q_init)

    # Simulation loop
    stockage["debit_sortie_kgh"] = stockage["valve_sortie_consigne"] * stockage["debit_max_kgh"]     # 1 = 100% ouverte
    
    if stockage["masse_stocked_kg"] + var_stk_in_q_kgh/3600 - stockage["debit_sortie_kgh"]/3600 < 0 : # masse négative
        tampon_debit_sortie_kgh = stockage["debit_sortie_kgh"] - (stockage["masse_stocked_kg"]*3600 + var_stk_in_q_kgh - stockage["debit_sortie_kgh"])      # Ajuste le débit
        tampon_valve_sortie_consigne = stockage["valve_sortie_consigne"]
        stockage = fct_stockage_init(p_init, v_init, q_init)
        stockage["debit_sortie_kgh"] = tampon_debit_sortie_kgh      # Ajuste le débit
        stockage["masse_stocked_kg"] = 0
        stockage["pression_bar"] = 1
        stockage["valve_sortie_consigne"] = tampon_valve_sortie_consigne

    else :
        stockage["masse_stocked_kg"] = stockage["masse_stocked_kg"] + var_stk_in_q_kgh/3600 - stockage["debit_sortie_kgh"]/3600    # Débit kgh to seconde
        stockage["delta_entalpie"] = stockage["delta_entalpie"] + stockage["debit_sortie_kgh"]/3600*stockage["entalpie_J"] - var_stk_in_q_kgh/3600*stockage["entalpie_J"]
        stockage["energie_interne_J"] = stockage["delta_entalpie"] / stockage["masse_stocked_kg"]
        stockage["masse_volumique_kgm3"] = stockage["masse_stocked_kg"] / stockage["volume_max_m3"]

        if stockage["masse_volumique_kgm3"] > 1 :                                                   # debug lorsque D = 31.46296973; value not found                D = 76.84 erreur aussi, round(x,1)
            stockage["masse_volumique_kgm3"] = round(stockage["masse_volumique_kgm3"],2)

        stockage["pression_bar"] = CoolProp.CoolProp.PropsSI('P','T',stockage["temp_kelv"],'D',stockage["masse_volumique_kgm3"],'hydrogen')/pow(10,5)
        stockage["temp_kelv"] = CoolProp.CoolProp.PropsSI('T','U',stockage["energie_interne_J"],'D',stockage["masse_volumique_kgm3"],'hydrogen')
        stockage["entalpie_J"] = -stockage["energie_interne_J"]

    return stockage

def fct_compressor(time_tick_sim_sec, compressor, source) :
    if time_tick_sim_sec == 1 :
        compressor = {"linéarisation_pts_1":(30,40),                        # Variable VARIABLE min (Pression,Débit kgh)
                        "linéarisation_pts_2":(100,75),                     # Variable VARIABLE (Pression,Débit kgh)
                        "debit_sortie_kgh":0,
                        "pression_sortie_bar":350,
                        "valve_sortie_consigne":0}
    
    if compressor["valve_sortie_consigne"] == 1 :               # Pas de vanne, pas de débit
        a = (compressor["linéarisation_pts_2"][1]-compressor["linéarisation_pts_1"][1])/(compressor["linéarisation_pts_2"][0]-compressor["linéarisation_pts_1"][0])
        b = compressor["linéarisation_pts_1"][1]-a*compressor["linéarisation_pts_1"][0]

        if source["pression_bar"] < compressor["linéarisation_pts_1"][0] :
            compressor_max_Q_H2_kgh = 0
        elif compressor["linéarisation_pts_1"][0] <= source["pression_bar"] < compressor["linéarisation_pts_2"][0] :
            compressor_max_Q_H2_kgh = a*source["pression_bar"]+b
        else :
            compressor_max_Q_H2_kgh = compressor["linéarisation_pts_2"][1]
        compressor["debit_sortie_kgh"] = compressor_max_Q_H2_kgh
    
    # source["debit_sortie_kgh"] = compressor["debit_sortie_kgh"]
    #  and source["valve_sortie_consigne"]
    return compressor

def fct_vanne(time_tick_sim_sec, vanne, debit_in_q_kgh) :
    if time_tick_sim_sec == 1 :
        vanne = {"debit_kgh_voie_1":0,
                "debit_kgh_voie_2":0,
                "debit_kgh_voie_3":0,
                "consigne_voie_1":0,
                "consigne_voie_2":0,
                "consigne_voie_3":0}

    if vanne["consigne_voie_1"] == 1 and vanne["consigne_voie_2"] == 0 and vanne["consigne_voie_3"] == 0 :
        vanne["debit_kgh_voie_1"] = debit_in_q_kgh
        vanne["debit_kgh_voie_2"] = 0
        vanne["debit_kgh_voie_3"] = 0
    elif vanne["consigne_voie_1"] == 0 and vanne["consigne_voie_2"] == 1 and vanne["consigne_voie_3"] == 0 :
        vanne["debit_kgh_voie_1"] = 0
        vanne["debit_kgh_voie_2"] = debit_in_q_kgh
        vanne["debit_kgh_voie_3"] = 0
    elif vanne["consigne_voie_1"] == 0 and vanne["consigne_voie_2"] == 0 and vanne["consigne_voie_3"] == 1 :
        vanne["debit_kgh_voie_1"] = 0
        vanne["debit_kgh_voie_2"] = 0
        vanne["debit_kgh_voie_3"] = debit_in_q_kgh
    elif vanne["consigne_voie_1"] == 0 and vanne["consigne_voie_2"] == 0 and vanne["consigne_voie_3"] == 0 :
        vanne["debit_kgh_voie_1"] = 0
        vanne["debit_kgh_voie_2"] = 0
        vanne["debit_kgh_voie_3"] = 0

    return vanne

def fct_echangeur(time_tick_sim_sec, echangeur, var_t_in_k, var_p_in_bar, var_q_in_kgh) :
    
    if time_tick_sim_sec == 1 :
        var_t_out_k = 273.15    -40                  # Variable VARIABLE
        var_seuil_declenchement_p_bar = 8            # Variable VARIABLE

        # pas de chute de pression et conservation des masses

        echangeur = {"temp_out_kelv":var_t_out_k,
                        "h_in":0,
                        "h_out":0,
                        "e_deleted_j":0,
                        "seuil_declenchement_p_bar":var_seuil_declenchement_p_bar,
                        "total_e_deleted_j":0}

    if var_p_in_bar >= echangeur["seuil_declenchement_p_bar"] :
        echangeur["h_in"] = CoolProp.CoolProp.PropsSI('H','P',var_p_in_bar*pow(10,5),'T',var_t_in_k,'H2')
        echangeur["h_out"] = CoolProp.CoolProp.PropsSI('H','P',var_p_in_bar*pow(10,5),'T',echangeur["temp_out_kelv"],'H2')
        echangeur["e_deleted_j"] = var_q_in_kgh/3600 * (echangeur["h_in"]-echangeur["h_out"])
    else :
        echangeur["h_in"] = 0
        echangeur["h_out"] = 0
        echangeur["e_deleted_j"] = 0
    echangeur["total_e_deleted_j"] = echangeur["total_e_deleted_j"] + echangeur["e_deleted_j"]

    return echangeur

# Initialisation
time_total_sim_sec  = 24*3600*7
time_tick_sim_sec   = 0

ELY_prod        = {}
stockage_BP     = {}
compressor_BP   = {}
vanne           = {}
stockage_MP_1   = {}
echangeur_1     = {}


# Loop main
while time_tick_sim_sec < time_total_sim_sec :
    time_tick_sim_sec = time_tick_sim_sec + 1

    ELY_prod        = fct_ELY_prod(time_tick_sim_sec, ELY_prod)
    stockage_BP     = fct_stockage(time_tick_sim_sec, stockage_BP, 1, 20, 18, ELY_prod["debit_kgh"])
    compressor_BP   = fct_compressor(time_tick_sim_sec, compressor_BP, stockage_BP)
    vanne           = fct_vanne(time_tick_sim_sec, vanne, compressor_BP["debit_sortie_kgh"])
    stockage_MP_1   = fct_stockage(time_tick_sim_sec, stockage_MP_1, 1, 6, 18,  vanne["debit_kgh_voie_1"])
    #echangeur_1     = fct_echangeur(1, echangeur_1, 293, 694, 180)


    # Commande
    if time_tick_sim_sec > 10000 :
        # vanne["consigne_voie_1"] = 1
        stockage_BP["valve_sortie_consigne"] = 1
        compressor_BP["valve_sortie_consigne"] = 1
        vanne["consigne_voie_1"] = 1



    if compressor_BP["valve_sortie_consigne"] == 1 :
        stockage_BP["debit_sortie_kgh"] = compressor_BP["debit_sortie_kgh"]

    if stockage_BP["pression_bar"] >= 40 :
        ELY_prod["valve_sortie_consigne"] = 0   # STOP de produire lorsque BP plein
        break



print("time : ",time_tick_sim_sec)
print("ELY débite : ", ELY_prod["total_h2_produced_kg"])
print("BP :     ", "Pression (Bar) : ",round(stockage_BP["pression_bar"],2), "   Masse (kg) : ",round(stockage_BP["masse_stocked_kg"],2), "   Temp (°C) : ",round(stockage_BP["temp_kelv"]-273.14,2))
print("MP :     ", "Pression (Bar) : ",round(stockage_MP_1["pression_bar"],2), "   Masse (kg) : ",round(stockage_MP_1["masse_stocked_kg"],2), "   Temp (°C) : ",round(stockage_MP_1["temp_kelv"]-273.14,2))

In [ ]:
import CoolProp


# CoolProp.CoolProp.PropsSI('T','D',57/1.8,'P',500*pow(10,5),'H2')

CoolProp.CoolProp.PropsSI('D','T',273.15+0,'P',30*pow(10,5),'H2')              # DMOLAR




0.08870773197410725

In [10]:
import CoolProp


# CoolProp.CoolProp.PropsSI('T','D',57/1.8,'P',500*pow(10,5),'H2')

CoolProp.CoolProp.PropsSI('D','T',273.15+15,'P',40*pow(10,5),'H2')*20              # DMOLAR



65.72435181914167

Pour MP&HP (300-500-900)
-fonction nature : eau utilisé / rejeté

-voir pour supprimer débit de sortie stockage

-Intégrer perte énergetique dans chaque dict ?
-fonction vide et fonction init avec masse initial

-Probleme de vanne et soustraction masse, faire une bonne commande
-Rajouter delta 20 bar mini pour flux
-Historique dic append temps + value

# Blocks

In [3]:
#Electrolyseur par la thermodynamique
import CoolProp

t_in_k = 273.15+15
t_out_k = 273.15+30
p_init_bar = 1
q_out_kgs = 18/3600
p_auxi_w = 0 # 185000

mmol_h2_kgPmol = CoolProp.CoolProp.PropsSI('M','H2')
mmol_O2_kgPmol = CoolProp.CoolProp.PropsSI('M','O2')
mmol_h20_kgPmol = CoolProp.CoolProp.PropsSI('M','water')

cp_mol_h2_jPkPmol = CoolProp.CoolProp.PropsSI('C','T',t_in_k,'P',p_init_bar*pow(10,5),'H2')*mmol_h2_kgPmol                  # J/kg/K * kg/mol
cp_mol_O2_jPkPmol = CoolProp.CoolProp.PropsSI('C','T',t_in_k,'P',p_init_bar*pow(10,5),'O2')*mmol_O2_kgPmol
cp_mol_h2O_jPkPmol = CoolProp.CoolProp.PropsSI('C','T',t_in_k,'P',p_init_bar*pow(10,5),'water')*mmol_h20_kgPmol

h_std_h2_jPmol = 0
h_std_o2_jPmol = 0
h_std_h2o_jPmol = -285830    #Enthalpie standard de formation H20 = -286 000J

rendement_ely = 0.60905

h_new_h2_jPmol = h_std_h2_jPmol + cp_mol_h2_jPkPmol*(t_out_k-t_in_k)
h_new_o2Pmol = h_std_o2_jPmol + cp_mol_O2_jPkPmol*(t_out_k-t_in_k)
h_new_h2oPmol = h_std_h2o_jPmol + cp_mol_h2O_jPkPmol*(t_out_k-t_in_k)

del_h_jPmol = h_new_h2_jPmol + 1/2*h_new_o2Pmol - h_new_h2oPmol

p_elec_w = q_out_kgs/mmol_h2_kgPmol * del_h_jPmol / rendement_ely
q_generated_w = p_elec_w * (1-rendement_ely)


# plug : 1 162 000 W
# cummin : 1 300 000 W
print("puissance totale : ",p_elec_w+p_auxi_w)
print("perte thermique : ",q_generated_w)

puissance totale :  1088854.377969651
perte thermique :  381099.0322893778


In [ ]:
# Save before changement init
def fct_stockage_BP(time_tick_sim_sec, stockage_BP, var_stk_in_q_kgh) :
    def fct_stockage_BP_init() :
        var_stk_temp_kelv = 273.14+20               # Variable VARIABLE 20 = T ambiant
        var_stk_p_bar = 1                           # Variable VARIABLE
        var_stk_masse_kg = 1/pow(10,6)                        # Variable VARIABLE
        var_stk_roh_kgm3 = 0                        # Variable VARIABLE Masse/MaxVolume
        var_stk_u_eint = CoolProp.CoolProp.PropsSI('U','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
        var_stk_h_ent = CoolProp.CoolProp.PropsSI('H','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
        var_stk_y_dan = var_stk_masse_kg*var_stk_u_eint
        var_stk_h_ent = -var_stk_u_eint

        stockage_BP_init = {"volume_max_m3":20,                     # Variable VARIABLE
                        "debit_max_kgh":18,                         # Variable VARIABLE
                        "masse_stocked_kg":var_stk_masse_kg,
                        "temp_kelv":var_stk_temp_kelv,
                        "pression_bar":var_stk_p_bar,
                        "masse_volumique_kgm3":var_stk_roh_kgm3,
                        "energie_interne_J":var_stk_u_eint,
                        "entalpie_J":var_stk_h_ent,
                        "delta_entalpie":var_stk_y_dan,
                        "valve_sortie_consigne":0,                  # Variable commande
                        "debit_sortie_kgh":0}
        return stockage_BP_init

    if time_tick_sim_sec == 1 :             # Initialise avec données de base puis change en fonction des entrées sorties
        stockage_BP = fct_stockage_BP_init()

    # Simulation loop
    stockage_BP["debit_sortie_kgh"] = stockage_BP["valve_sortie_consigne"] * stockage_BP["debit_max_kgh"]     # 1 = 100% ouverte
    
    if stockage_BP["masse_stocked_kg"] + var_stk_in_q_kgh/3600 - stockage_BP["debit_sortie_kgh"]/3600 < 0 : # masse négative
        tampon_debit_sortie_kgh = stockage_BP["debit_sortie_kgh"] - (stockage_BP["masse_stocked_kg"]*3600 + var_stk_in_q_kgh - stockage_BP["debit_sortie_kgh"])      # Ajuste le débit
        tampon_valve_sortie_consigne = stockage_BP["valve_sortie_consigne"]
        stockage_BP = fct_stockage_BP_init()
        stockage_BP["debit_sortie_kgh"] = tampon_debit_sortie_kgh      # Ajuste le débit
        stockage_BP["masse_stocked_kg"] = 0
        stockage_BP["pression_bar"] = 1
        stockage_BP["valve_sortie_consigne"] = tampon_valve_sortie_consigne

    else :
        stockage_BP["masse_stocked_kg"] = stockage_BP["masse_stocked_kg"] + var_stk_in_q_kgh/3600 - stockage_BP["debit_sortie_kgh"]/3600    # Débit kgh to seconde
        stockage_BP["delta_entalpie"] = stockage_BP["delta_entalpie"] + stockage_BP["debit_sortie_kgh"]/3600*stockage_BP["entalpie_J"] - var_stk_in_q_kgh/3600*stockage_BP["entalpie_J"]
        stockage_BP["energie_interne_J"] = stockage_BP["delta_entalpie"] / stockage_BP["masse_stocked_kg"]
        stockage_BP["masse_volumique_kgm3"] = stockage_BP["masse_stocked_kg"] / stockage_BP["volume_max_m3"]
        stockage_BP["pression_bar"] = CoolProp.CoolProp.PropsSI('P','T',stockage_BP["temp_kelv"],'D',stockage_BP["masse_volumique_kgm3"],'hydrogen')/pow(10,5)
        stockage_BP["temp_kelv"] = CoolProp.CoolProp.PropsSI('T','U',stockage_BP["energie_interne_J"],'D',stockage_BP["masse_volumique_kgm3"],'hydrogen')
        stockage_BP["entalpie_J"] = -stockage_BP["energie_interne_J"]

    return stockage_BP


In [ ]:
# ELY Fonction

def fct_ELY_prod(time_tick_sim_sec, ELY_prod) :
    if time_tick_sim_sec == 1 :
        ELY_prod = {"tension_cell_v":2.7,
                    "puissance_kw":1300,
                    "pression_bar":30,
                    "energie_spent_kwh":0,
                    "debit_kgh":0,
                    "valve_sortie_consigne":1}

    # Simulation loop
    
    ELY_prod["debit_kgh"] = ELY_prod["valve_sortie_consigne"] * ELY_prod["puissance_kw"]*pow(10,3) / (2*96485*ELY_prod["tension_cell_v"]) * 2.02*3.6    # 2.02 g/mol       0.0899 kg/Nm3       3.6 = s to h & g to kg

    ELY_prod["energie_spent_kwh"] = ELY_prod["energie_spent_kwh"] + ELY_prod["valve_sortie_consigne"]*ELY_prod["puissance_kw"]/3600

    return ELY_prod


time_total_sim_sec = 3600
time_tick_sim_sec = 0

ELY_prod = {}
ELY_total_kg_produced = 0

while time_tick_sim_sec < time_total_sim_sec :
    time_tick_sim_sec = time_tick_sim_sec + 1

    ELY_prod = fct_ELY_prod(time_tick_sim_sec, ELY_prod)
    ELY_total_kg_produced = ELY_total_kg_produced + ELY_prod["debit_kgh"]/3600



print("H2 prod (kg)           : ", ELY_total_kg_produced)

ELY_prod

In [ ]:
# Stockage H2 fonction
import CoolProp

def fct_stockage_BP(time_tick_sim_sec, stockage_BP, var_stk_in_q_kgh) :
    def fct_stockage_BP_init() :
        var_stk_temp_kelv = 273.14+20               # Variable VARIABLE 20 = T ambiant
        var_stk_p_bar = 1                           # Variable VARIABLE
        var_stk_v_max_m3 = 5.8                          # Variable VARIABLE

        var_stk_masse_kg = 1/pow(10,6)                        # Variable VARIABLE
        var_stk_roh_kgm3 = var_stk_masse_kg/var_stk_v_max_m3
        var_stk_u_eint = CoolProp.CoolProp.PropsSI('U','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
        var_stk_h_ent = CoolProp.CoolProp.PropsSI('H','T',var_stk_temp_kelv,'P',var_stk_p_bar*pow(10,5),'hydrogen')
        var_stk_y_dan = var_stk_masse_kg*var_stk_u_eint
        var_stk_h_ent = -var_stk_u_eint

        stockage_BP_init = {"volume_max_m3":var_stk_v_max_m3,                     # Variable VARIABLE
                        "debit_max_kgh":18,                         # Variable VARIABLE
                        "masse_stocked_kg":var_stk_masse_kg,
                        "temp_kelv":var_stk_temp_kelv,
                        "pression_bar":var_stk_p_bar,
                        "masse_volumique_kgm3":var_stk_roh_kgm3,
                        "energie_interne_J":var_stk_u_eint,
                        "entalpie_J":var_stk_h_ent,
                        "delta_entalpie":var_stk_y_dan,
                        "valve_sortie_consigne":0,                  # Variable commande
                        "debit_sortie_kgh":0}
        return stockage_BP_init

    if time_tick_sim_sec == 1 :             # Initialise avec données de base puis change en fonction des entrées sorties
        stockage_BP = fct_stockage_BP_init()

    # Simulation loop
    stockage_BP["debit_sortie_kgh"] = stockage_BP["valve_sortie_consigne"] * stockage_BP["debit_max_kgh"]     # 1 = 100% ouverte
    
    if stockage_BP["masse_stocked_kg"] + var_stk_in_q_kgh/3600 - stockage_BP["debit_sortie_kgh"]/3600 < 0 : # masse négative
        tampon_debit_sortie_kgh = stockage_BP["debit_sortie_kgh"] - (stockage_BP["masse_stocked_kg"]*3600 + var_stk_in_q_kgh - stockage_BP["debit_sortie_kgh"])      # Ajuste le débit
        tampon_valve_sortie_consigne = stockage_BP["valve_sortie_consigne"]
        stockage_BP = fct_stockage_BP_init()
        stockage_BP["debit_sortie_kgh"] = tampon_debit_sortie_kgh      # Ajuste le débit
        stockage_BP["masse_stocked_kg"] = 0
        stockage_BP["pression_bar"] = 1
        stockage_BP["valve_sortie_consigne"] = tampon_valve_sortie_consigne

    else :
        stockage_BP["masse_stocked_kg"] = stockage_BP["masse_stocked_kg"] + var_stk_in_q_kgh/3600 - stockage_BP["debit_sortie_kgh"]/3600    # Débit kgh to seconde
        stockage_BP["delta_entalpie"] = stockage_BP["delta_entalpie"] + stockage_BP["debit_sortie_kgh"]/3600*stockage_BP["entalpie_J"] - var_stk_in_q_kgh/3600*stockage_BP["entalpie_J"]
        stockage_BP["energie_interne_J"] = stockage_BP["delta_entalpie"] / stockage_BP["masse_stocked_kg"]
        stockage_BP["masse_volumique_kgm3"] = stockage_BP["masse_stocked_kg"] / stockage_BP["volume_max_m3"]
        stockage_BP["pression_bar"] = CoolProp.CoolProp.PropsSI('P','T',stockage_BP["temp_kelv"],'D',stockage_BP["masse_volumique_kgm3"],'hydrogen')/pow(10,5)
        stockage_BP["temp_kelv"] = CoolProp.CoolProp.PropsSI('T','U',stockage_BP["energie_interne_J"],'D',stockage_BP["masse_volumique_kgm3"],'hydrogen')
        stockage_BP["entalpie_J"] = -stockage_BP["energie_interne_J"]

    return stockage_BP


time_total_sim_sec = 3600
time_tick_sim_sec = 0

stockage_BP = {}

var_stk_in_q_kgh = 180
valve_opened = False

while time_tick_sim_sec < time_total_sim_sec :
    time_tick_sim_sec = time_tick_sim_sec + 1

    stockage_BP = fct_stockage_BP(time_tick_sim_sec, stockage_BP, var_stk_in_q_kgh)

    if time_tick_sim_sec == 1 :
        print("masse (kg) : ", stockage_BP["masse_stocked_kg"], "pressure (Bar) : ", stockage_BP["pression_bar"], "température (°C) : ", stockage_BP["temp_kelv"]-273.15)
        print(stockage_BP)

print("masse (kg) : ", stockage_BP["masse_stocked_kg"], "pressure (Bar) : ", stockage_BP["pression_bar"], "température (°C) : ", stockage_BP["temp_kelv"]-273.15)
stockage_BP

# Simu disponibilité

In [1]:
# 2 ELY parallelle 96% dispo chacun
import random

def sim_a_year() :
    days = 0
    panne_counter = 0

    while days < 365 :
        days = days + 1

        ely_1_status = random.randint(0,100)
        ely_2_status = random.randint(0,100)

        if ely_1_status > 96 and ely_2_status > 96 :
            panne_counter = panne_counter + 1
    return panne_counter

years = 0
total_pannes = 0
echantillon = 10000

while years < echantillon :
    years = years + 1
    total_pannes = total_pannes + sim_a_year()

print("Jour indisponible sur {} ans             : ".format(echantillon),total_pannes)
print("Moyenne jour indisponible sur 1 an       : ",total_pannes/echantillon)
print("Disponibilité de 2 stations parallelle   : ",(1-total_pannes / (365*echantillon))*100)

Jour indisponible sur 10000 ans             :  5833
Moyenne jour indisponible sur 1 an       :  0.5833
Disponibilité de 2 stations parallelle   :  99.84019178082193


In [1]:
# 3 ELY parallelle 96% dispo chacun mais besoin de 2 mini
import random

def sim_a_year() :
    days = 0
    panne_counter = 0

    while days < 365 :
        days = days + 1

        ely_1_status = random.randint(0,100)
        ely_2_status = random.randint(0,100)
        ely_3_status = random.randint(0,100)

        if ely_1_status > 96 and ely_2_status > 96 or ely_1_status > 96 and ely_3_status > 96 or ely_2_status > 96 and ely_3_status > 96 :
            panne_counter = panne_counter + 1
    return panne_counter


years = 0
total_pannes = 0
echantillon = 10000

while years < echantillon :
    years = years + 1
    total_pannes = total_pannes + sim_a_year()

print("Jour indisponible sur {} ans             : ".format(echantillon),total_pannes)
print("Moyenne jour indisponible sur 1 an       : ",total_pannes/echantillon)
print("Disponibilite de 2 stations parallele   : ",(1-total_pannes / (365*echantillon))*100)

Jour indisponible sur 10000 ans             :  16767
Moyenne jour indisponible sur 1 an       :  1.6767
Disponibilite de 2 stations parallele   :  99.5406301369863


In [11]:

import random

def sim_a_year() :
    days = 0
    panne_counter_1 = 0
    panne_counter_2 = 0
    panne_counter_3 = 0
    panne_counter_4 = 0
    panne_counter_5 = 0

    while days < 365 :
        panned_counter_intern = 0

        days = days + 1

        ely_1_status = random.randint(0,100)
        ely_2_status = random.randint(0,100)
        ely_3_status = random.randint(0,100)
        ely_4_status = random.randint(0,100)
        ely_5_status = random.randint(0,100)

        # check de panne simultané
        if ely_1_status > 96 :
            panned_counter_intern = panned_counter_intern + 1
        if ely_2_status > 96 :
            panned_counter_intern = panned_counter_intern + 1
        if ely_3_status > 96 :
            panned_counter_intern = panned_counter_intern + 1 
        if ely_4_status > 96 :
            panned_counter_intern = panned_counter_intern + 1 
        if ely_5_status > 96 :
            panned_counter_intern = panned_counter_intern + 1  
        

        if panned_counter_intern == 1 :
            panne_counter_1 = panne_counter_1 + 1
        if panned_counter_intern == 2 :
            panne_counter_2 = panne_counter_2 + 1
        if panned_counter_intern == 3 :
            panne_counter_3 = panne_counter_3 + 1
        if panned_counter_intern == 4 :
            panne_counter_4 = panne_counter_4 + 1
        if panned_counter_intern == 5 :
            panne_counter_5 = panne_counter_5 + 1


    return panne_counter_1, panne_counter_2, panne_counter_3, panne_counter_4, panne_counter_5




years = 0
total_pannes_1 = 0
total_pannes_2 = 0
total_pannes_3 = 0
total_pannes_4 = 0
total_pannes_5 = 0
echantillon = 10000

while years < echantillon :
    years = years + 1
    panne_counter_1, panne_counter_2, panne_counter_3, panne_counter_4, panne_counter_5 = sim_a_year()

    total_pannes_1 = total_pannes_1 + panne_counter_1
    total_pannes_2 = total_pannes_2 + panne_counter_2
    total_pannes_3 = total_pannes_3 + panne_counter_3
    total_pannes_4 = total_pannes_4 + panne_counter_4
    total_pannes_5 = total_pannes_5 + panne_counter_5


print("Simulation faite sur {} ans             : ".format(echantillon))

print("1 ely en panne : ",total_pannes_1)
print("2 ely en panne : ",total_pannes_2)
print("3 ely en panne : ",total_pannes_3)
print("4 ely en panne : ",total_pannes_4)
print("5 ely en panne : ",total_pannes_5)

print("1 ely en panne dispo % : ",(1-total_pannes_1 / (365*echantillon))*100)
print("2 ely en panne dispo % : ",(1-total_pannes_2 / (365*echantillon))*100)
print("3 ely en panne dispo % : ",(1-total_pannes_3 / (365*echantillon))*100)
print("4 ely en panne dispo % : ",(1-total_pannes_4 / (365*echantillon))*100)
print("5 ely en panne dispo % : ",(1-total_pannes_5 / (365*echantillon))*100)



# print("Jour indisponible sur {} ans             : ".format(echantillon),total_pannes)
# print("Moyenne jour indisponible sur 1 an       : ",total_pannes/echantillon)
# print("Disponibilité de 2 stations parallelle   : ",(1-total_pannes / (365*echantillon))*100)

Simulation faite sur 10000 ans             : 
1 ely en panne :  615828
2 ely en panne :  50684
3 ely en panne :  2026
4 ely en panne :  48
5 ely en panne :  2
1 ely en panne dispo % :  83.128
2 ely en panne dispo % :  98.61139726027398
3 ely en panne dispo % :  99.94449315068493
4 ely en panne dispo % :  99.99868493150686
5 ely en panne dispo % :  99.99994520547946
